In [1]:
] activate

 Activating environment at `~/cache/DCSideBatteryModeling/Project.toml`


In [2]:
using LinearAlgebra

using OrdinaryDiffEq #Gets the solvers
using Plots
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

In [3]:
plotly()

Plots.PlotlyBackend()

In [4]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /Users/jdlara/.julia/packages/InfrastructureSystems/uXGZ2/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:418
┌ Info: Reading branch data
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/parsers/power_models_data.jl:575
┌ Info: Reading branch da

In [5]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /Users/jdlara/.julia/packages/InfrastructureSystems/uXGZ2/src/in_memory_time_series_storage.jl:24
┌ Warning: There are no Generator Components in the System
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/utils/IO/system_checks.jl:56
┌ Warning: System peak load (50.0) exceeds total capacity capability (0).
└ @ PowerSystems /Users/jdlara/.julia/packages/PowerSystems/5TSyV/src/utils/IO/system_checks.jl:72


,ConcreteType,SuperTypes
,String,String
1,Area,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
2,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
3,"DynamicInverter{AverageConverter,OuterControl{VirtualInertia,ReactivePowerDroop},CurrentControl,FixedDCSource,KauraPLL,LCLFilter}",DynamicInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
4,GenericBattery,Storage <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
5,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
6,LoadZone,AggregationTopology <: Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any


## Instantiate 0th-order ODE Model

In [6]:
parameter_mapping = instantiate_parameters(omib_sys)
M= instantiate_0th_order_model(omib_sys)

ModelOperatingPoint


In [7]:
_parameter_values = [x.second for x in parameter_mapping] # Extract list of parameter values

50-element Array{Float64,1}:
  376.99111843077515
    0.2
    0.01
    0.2
    0.0
    1.01
    1.0
    1.0
    0.2
    0.08
    0.074
    0.003
   37.69911184307752
    ⋮
    1.084409521544636
    0.6
    4.0
    0.3863
   10.34
    0.01
   -6.0
  -12.0
  -12.0
  -60.0
 -120.0
    0.0003125

In [8]:
fb = 60 # Base frequency. 
_ωb = 2 * pi * fb # Base angular velocity 
Sb = 0.2e6 # Base Power
# System base for AC side
Vb = 480 # Base AC voltage
Ib = Sb / Vb # Base AC Current 
Zb = Vb / Ib # Base Impedancec
Lb = Zb / _ωb # Base Inductance
Cb = 1 / (Zb * _ωb) # Base AC capcitacne
# System base for DC side
Vb_dc = (sqrt(8)/sqrt(3))*Vb # Base DC voltage
Ib_dc = Sb / Vb_dc # Base DC Current
Zb_dc = Vb_dc / Ib_dc # Base DC Impedance
Lb_dc = Zb_dc / _ωb # Base DC Inductance
Cb_dc = 1 / (Zb_dc * _ωb) # Base DC Capcitance

0.0008634708284065503

## Build Jacobian Function

In [9]:
jac_exp = get_0th_order_jacobian_expression()
_jac = eval(jac_exp)

#48 (generic function with 1 method)

In [10]:
jac_eval = (out, u0, params) -> _jac(out, u0, params)
param_eval = (out, params) -> _jac(out, M.u0, params)
n = length(M.u0)
J = zeros(n, n);

In [11]:
M(_parameter_values)
param_eval(J, _parameter_values)
jac = ModelJacobian(jac_eval, J)

Jacobian


## Identfying Set of Small-Signal Stable Gains

In [ ]:
cap_range = 2000:500:7000 # Looping throught different DC-link capacitor sizing
param_range = 0.5:0.5:10.5 # Looping throughc control  
_parameter_values[44]=0 # Setting K_pred=0
max_eig = -3 # Design specification - Maximum real part of eigenvalues
min_damping = 0.35 # Design specification - Minimum damping
for cap_size in cap_range
    _parameter_values[38]=cap_size*1e-6/Cb_dc # Assigning DC-link capacitor size
    stable_gains = reshape([],0,4) # Initializing array of stable gains
    for kpv in param_range
        _parameter_values[40]=kpv # K_p^v
        for kiv in param_range  
            _parameter_values[41]=kiv #K_i^v
            for kpi in param_range
                _parameter_values[42]=kpi #K_p^i
                for kii in param_range
                    _parameter_values[43]=kii #K_i^i
                    M(_parameter_values) # Re-calculating operating point 
                    param_eval(J, _parameter_values) 
                    jac = ModelJacobian(jac_eval, J)
                    eig_vals = eigvals(jac(M)) # Calculating eigenvalues of linearized system
                    damp = [-1*real(λ) / sqrt(real(λ)^2 + imag(λ)^2) for λ in eig_vals] # Calcualting damping ratios
                    if maximum(real(eig_vals)) < max_eig && minimum(damp)>min_damping # Checking to see if gains satisfy design criteria
                        stable_gains = [stable_gains; [kpv kiv kpi kii ]] # Concatenating stable gains
                    end
                end 

            end

        end

    end
    fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
    writedlm( fileName,  stable_gains, ',') # Writing array of gains that satisfy design criteria 
end

## Searching Over Set of Stable Gains for Large Signal Tuning

In [ ]:
cap_range = 2000:500:7000 # Looping throught different DC-link capacitor sizing
_parameter_values[44]=0 #K_pred
for cap_size in cap_range
    _parameter_values[38]=cap_size*1e-6/Cb_dc # Assigning DC-link capacitor size
    fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
    gains = readdlm(fileName, ',', Float64) # Loading array of Stable Gains
#     fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
#     optIndex = readdlm(fileName, ',', Float64)
    optIndex = [1, 1e6] # Initializing optIndex (index 1) and associated l_2 norm values (index 2)
    for i in 1:1:1000
        index = rand(1:size(gains,1))
        _parameter_values[40]=gains[index,1] #K_p^v
        _parameter_values[41]=gains[index,2] #K_i^v
        _parameter_values[42]=gains[index,3] #K_p^i
        _parameter_values[43]=gains[index,4] #K_i^i
        M(_parameter_values) #Re-calcualte operating point

        disturbance=deepcopy(_parameter_values)
        disturbance[9]=0.7

        sim = instantiate_0th_order_dae(disturbance, M; tspan = (0.0, 0.02))
        sol = solve(sim, Rodas5(), saveat = 0.001);
        solArray = Array(sol)
        global runError=norm(_parameter_values[39].-solArray[13,:]) # Calculate l2 norm error
        if runError < optIndex[2] # If l_2 norm error is less that previously stored
            optIndex[1]=index  # Store current array index as optimal index
            optIndex[2]=runError
            fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
            writedlm(fileName,  optIndex, ',') #Write csv with index and l_2 norm error
        end
    end
end